#                                                                      Toronto Project
<img src="https://media.giphy.com/media/iuuwqzPEDAZqM/giphy.gif" width="500" height="500" />


## Libraries

In [1]:
import pandas as pd 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import numpy as np
#webscaper
!conda install -c conda-forge beautifulsoup4 --yes
from bs4 import BeautifulSoup
#htmlparsers
import requests
import folium
import matplotlib.pyplot as plt


Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.7.12

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs: 
    - beautifulsoup4


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    soupsieve-1.9.4            |           py36_0          58 KB  conda-forge
    beautifulsoup4-4.8.1       |           py36_0         149 KB  conda-forge
    openssl-1.1.1d             |       h516909a_0         2.1 MB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.3 MB

The following NEW packages will be INSTALLED:

    soupsieve:      1.9.4-py36_0      conda-forge

The following packages will be UPDATED:

    beautiful

## Web Scraping 

In [2]:

page =requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
soup = BeautifulSoup(page.content,'html.parser')


In [3]:
data = []
postcodes=[]
borough = []
neighbourhood=[]
table =soup.table
for tr in table.find_all('tr'):
    data.append(tr.text)
for i in range(len(data)):    
    data[i] = data[i].split('\n')
for i in data:
    del i[0]
    del i[-1]
for i in data:
    postcodes.append(i[0])
    borough.append(i[1])
    neighbourhood.append(i[2])
del postcodes[0]
del borough[0] 
del neighbourhood[0]
d = {'PostalCode': postcodes, 'Borough':borough, 'Neighborhood':neighbourhood}
toronto_df = pd.DataFrame(data=d)
toronto_df = toronto_df[toronto_df.Borough !='Not assigned']
toronto_df.loc[[8],['Neighborhood']]="Queen's Park"
#toronto_df.reset_index(drop=True)

## Clean up

In [4]:
toronto_pc = toronto_df.PostalCode.tolist()
def unique(list1):
    unique_list = []
    for x in list1:
        if x not in unique_list:
            unique_list.append(x)
    return(unique_list)
toronto_pc = unique(toronto_pc)
temp_n=[]
temp_b=[]
for i in range(len(toronto_pc)):
    temp_n.append(toronto_df.loc[toronto_df['PostalCode']==toronto_pc[i]]['Neighborhood'].values)
    temp_b.append(unique(toronto_df.loc[toronto_df['PostalCode']==toronto_pc[i]]['Borough'].values.tolist()))
d={'PostalCode': toronto_pc, 'Borough':temp_b, 'Neighborhood':temp_n}
toronto_df = pd.DataFrame(data=d)

In [5]:

toronto_df['Borough']= toronto_df['Borough'].astype(str)
toronto_df['Neighborhood']= toronto_df['Neighborhood'].astype(str)
toronto_df['Borough'] = toronto_df['Borough'].str.strip('[]')
toronto_df['Borough'] = toronto_df['Borough'].str.strip("''")
toronto_df['Neighborhood'] = toronto_df['Neighborhood'].str.strip('[]')
toronto_df['Neighborhood'] = toronto_df['Neighborhood'].str.strip("''")
toronto_df.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront' 'Regent Park
3,M6A,North York,Lawrence Heights' 'Lawrence Manor
4,M7A,"""Queen's Park""","""Queen's Park"""


## Postal Code Merge 

In [6]:
coordinates = pd.read_csv("Geospatial_Coordinates.csv")
coordinates.rename(columns={'Postal Code':"PostalCode"}, inplace=True)
coordinates.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [7]:
toronto_df = toronto_df.merge(coordinates,on='PostalCode')
toronto_df.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,Harbourfront' 'Regent Park,43.654260,-79.360636
3,M6A,North York,Lawrence Heights' 'Lawrence Manor,43.718518,-79.464763
4,M7A,"""Queen's Park""","""Queen's Park""",43.662301,-79.389494


## Create a map of Toronto 


In [8]:

data='simple.geojson'

m = folium.Map(location=[43.6532,-79.3832], zoom_start=11)
folium.GeoJson(data,name="geojson").add_to(m)
for lat,lng, postalc ,borough  in zip(toronto_df['Latitude'], toronto_df['Longitude'], toronto_df['PostalCode'],toronto_df['Borough']):
    label= 'Postal Code: {}, Borough: {}'.format(postalc,borough)
    label = folium.Popup(label,parse_html=True)
    folium.CircleMarker(
    [lat,lng],
    radius=5,
    popup=label,
    color='black',
    fill=True,
    fill_color='#000000',
    fill_opacity=0.7,parse_html=False
    ).add_to(m)
#m.choropleth(
#geo_data=data)
#folium.LayerControl().add_to(m)
m

## Foursquare API

In [17]:
CLIENT_ID = '********************************************'
CLIENT_SECRET = '*************************************'
VERSION = '20180605'
radius =500
LIMIT=100

In [30]:
def getNearbyVenues(pcs, boroughs, neighborhoods, latitude, longitude,radius=500):
    venues_list=[]
    for pc, boro, neighbor, lat,lng in zip(pcs, boroughs, neighborhoods,latitude,longitude):
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
        results =requests.get(url).json()
        results = results['response']['groups'][0]['items']
        venues_list.append([(pc,boro,neighbor,lat,lng,v['venue']['name'],
                             v['venue']['location']['lat'],
                             v['venue']['location']['lng'],
                             v['venue']['categories'][0]['name']) for v in results])
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns=['PostalCode','Borough', 'Neighborhoods',
                  'Borough Latitude', 
                  'Borough Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    return(nearby_venues)
Toronto_Venues = getNearbyVenues(pcs=toronto_df['PostalCode'],boroughs=toronto_df['Borough'], neighborhoods=toronto_df['Neighborhood'], latitude=toronto_df['Latitude'], longitude=toronto_df['Longitude'])

In [36]:
Toronto_Venues.shape 
Toronto_Venues.head()

,PostalCode,Borough,Neighborhoods,Borough Latitude,Borough Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,M3A,North York,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,M3A,North York,Parkwoods,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
2,M4A,North York,Victoria Village,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena
3,M4A,North York,Victoria Village,43.725882,-79.315572,Tim Hortons,43.725517,-79.313103,Coffee Shop
4,M4A,North York,Victoria Village,43.725882,-79.315572,Portugril,43.725819,-79.312785,Portuguese Restaurant


In [34]:
Toronto_Venues.to_csv('Toronto_Venues')

In [38]:
print('There are {} unique categories'.format(len(Toronto_Venues['Venue Category'].unique())))

There are 271 unique categories


## Onehot encoding

In [46]:
toronto_onehot = pd.get_dummies(Toronto_Venues[['Venue Category']],prefix='',prefix_sep='')
toronto_onehot['PostalCode']=Toronto_Venues['PostalCode']
fixed_columns= [toronto_onehot.columns[-1]]+list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]
toronto_onehot.head()

,PostalCode,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Fabric Shop,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Luggage Store,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Motel,Movie Theater,Moving Target,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Paper / Office Supplies Store,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,River,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Social Club,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,M3A,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [47]:
toronto_onehot.shape

(2235, 272)

In [49]:
toronto_grouped=toronto_onehot.groupby('PostalCode').mean().reset_index()
toronto_grouped.head()

,PostalCode,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Fabric Shop,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Luggage Store,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Motel,Movie Theater,Moving Target,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Paper / Office Supplies Store,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,River,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Social Club,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,M1B,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.00000

## Ranking Popularity in each PC


In [63]:
#put into pandas df
def return_most_common_venue(row,num_top_venues):
    row_categories=row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    return row_categories_sorted.index.values[0:num_top_venues]

num_top_venues = 10 
indicators = ['st','nd', 'rd']
columns = ['PostalCode']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))
        
pc_venues_sorted = pd.DataFrame(columns=columns)
pc_venues_sorted['PostalCode']= toronto_grouped['PostalCode']

for ind in np.arange(toronto_grouped.shape[0]):
    pc_venues_sorted.iloc[ind,1:]= return_most_common_venue(toronto_grouped.iloc[ind,:], num_top_venues)
    
pc_venues_sorted.head()

,PostalCode,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Fast Food Restaurant,Dumpling Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Yoga Studio,Dim Sum Restaurant
1,M1C,Moving Target,Bar,Dumpling Restaurant,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Yoga Studio,Diner
2,M1E,Electronics Store,Pizza Place,Intersection,Breakfast Spot,Medical Center,Rental Car Location,Mexican Restaurant,Yoga Studio,Donut Shop,Dog Run
3,M1G,Coffee Shop,Korean Restaurant,Yoga Studio,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store
4,M1H,Hakka Restaurant,Bakery,Bank,Athletics & Sports,Thai Restaurant,Caribbean Restaurant,Fried Chicken Joint,Discount Store,Doner Restaurant,Donut Shop


## Clustering 

In [66]:
from sklearn.cluster import KMeans
Kcluster =5
toronto_grouped_clustering = toronto_grouped.drop('PostalCode', 1)
#running kmeans
kmeans = KMeans(n_clusters=Kcluster, random_state=0).fit(toronto_grouped_clustering)
kmeans.labels_

array([1, 4, 4, 1, 4, 2, 1, 4, 1, 4, 4, 1, 4, 1, 2, 1, 1, 1, 4, 3, 4, 1,
       0, 1, 0, 4, 1, 1, 1, 0, 4, 4, 4, 1, 1, 4, 4, 1, 1, 0, 1, 1, 1, 4,
       1, 1, 1, 2, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 4, 4, 1, 4,
       1, 1, 1, 1, 1, 4, 4, 0, 4, 4, 1, 1, 4, 1, 1, 4, 1, 1, 1, 1, 1, 1,
       1, 0, 4, 1, 1, 1, 4, 0, 1, 1, 1, 4], dtype=int32)

In [67]:
#add labels 
pc_venues_sorted.insert(0,'Cluster Labels',kmeans.labels_)
toronto_merged = toronto_df
toronto_merged = toronto_merged.join(pc_venues_sorted.set_index('PostalCode'), on='PostalCode')
toronto_merged.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.753259,-79.329656,0.0,Food & Drink Shop,Park,Yoga Studio,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant
1,M4A,North York,Victoria Village,43.725882,-79.315572,1.0,Pizza Place,Coffee Shop,Hockey Arena,Portuguese Restaurant,Intersection,Empanada Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Dessert Shop
2,M5A,Downtown Toronto,Harbourfront' 'Regent Park,43.654260,-79.360636,1.0,Coffee Shop,Park,Bakery,Theater,Breakfast Spot,Restaurant,Café,Pub,Mexican Restaurant,Yoga Studio
3,M6A,North York,Lawrence Heights' 'Lawrence Manor,43.718518,-79.464763,1.0,Clothing Store,Furniture / Home Store,Accessories Store,Coffee Shop,Event Space,Miscellaneous Shop,Arts & Crafts Store,Boutique,Vietnamese Restaurant,Coworking Space
4,M7A,"""Queen's Park""","""Queen's Park""",43.662301,-79.389494,1.0,Coffee Shop,Park,Gym,Diner,Yoga Studio,Burrito Place,Italian Restaurant,Japanese Restaurant,Nightclub,Seafood Restaurant


In [96]:
#visualize the map
toronto_merged=toronto_merged.dropna(axis='rows')
toronto_merged=toronto_merged.astype({'Cluster Labels':'int32'})
import matplotlib.cm as cm
import matplotlib.colors as colors 
m = folium.Map(location=[43.6532,-79.3832], zoom_start=11)
folium.GeoJson(data,name="geojson").add_to(m)
x = np.arange(Kcluster)
ys= [i+x +(i*x)**2 for i in range(Kcluster)]
color_array= cm.rainbow(np.linspace(0,1,len(ys)))
rainbow = [colors.rgb2hex(i) for i in color_array]
markers_colors = []
for lat,lng, postalc ,borough, cluster  in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['PostalCode'],toronto_merged['Borough'],toronto_merged['Cluster Labels']):
    label= 'Postal Code: {}, Borough: {}, Cluster Label {}'.format(postalc,borough,cluster)
    label = folium.Popup(label,parse_html=True)
    folium.CircleMarker(
    [lat,lng],
    radius=5,
    popup=label,
    color=rainbow[cluster-1],
    fill=True,
    fill_color=rainbow[cluster-1],
    fill_opacity=0.7,parse_html=False
    ).add_to(m)
m

## Examine the clusters


In [104]:
#Custer 0
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[0] + list(range(5, toronto_merged.shape[1]))]]

,PostalCode,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,0,Food & Drink Shop,Park,Yoga Studio,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant
21,M6E,0,Park,Fast Food Restaurant,Women's Store,Market,Dumpling Restaurant,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore
35,M4J,0,Park,Convenience Store,Yoga Studio,Eastern European Restaurant,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant
40,M3K,0,Airport,Park,Yoga Studio,Eastern European Restaurant,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant
64,M9N,0,Park,Yoga Studio,Eastern European Restaurant,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Electronics Store
66,M2P,0,Park,Bank,Convenience Store,Yoga Studio,Eastern European Restaurant,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant
91,M4W,0,Park,Playground,Trail,Yoga Studio,Dumpling Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop
98,M8X,0,Park,River,Yoga Studio,Drugstore,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant


In [105]:
#Cluster 1

toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[0] + list(range(5, toronto_merged.shape[1]))]]

,PostalCode,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,M4A,1,Pizza Place,Coffee Shop,Hockey Arena,Portuguese Restaurant,Intersection,Empanada Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Dessert Shop
2,M5A,1,Coffee Shop,Park,Bakery,Theater,Breakfast Spot,Restaurant,Café,Pub,Mexican Restaurant,Yoga Studio
3,M6A,1,Clothing Store,Furniture / Home Store,Accessories Store,Coffee Shop,Event Space,Miscellaneous Shop,Arts & Crafts Store,Boutique,Vietnamese Restaurant,Coworking Space
4,M7A,1,Coffee Shop,Park,Gym,Diner,Yoga Studio,Burrito Place,Italian Restaurant,Japanese Restaurant,Nightclub,Seafood Restaurant
6,M1B,1,Fast Food Restaurant,Dumpling Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Yoga Studio,Dim Sum Restaurant
8,M4B,1,Pizza Place,Fast Food Restaurant,Intersection,Breakfast Spot,Athletics & Sports,Gastropub,Bus Line,Gym / Fitness Center,Pharmacy,Pet Store
9,M5B,1,Clothing Store,Coffee Shop,Cosmetics Shop,Café,Bakery,Theater,Burger Joint,Pizza Place,Bubble Tea Shop,Sporting Goods Shop
13,M3C,1,Beer Store,Gym,Coffee Shop,Concert Hall,Chinese Restaurant,Fast Food Restaurant,Bike Shop,Sporting Goods Shop,Sandwich Place,Japanese Restaurant
15,M5C,1,Coffee Shop,Café,Hotel,Restaurant,Italian Restaurant,Clothing Store,Cosmetics Shop,Breakfast Spot,Bakery,Gastropub
17,M9C,1,Pharmacy,Coffee Shop,Park,Liquor Store,Beer Store,Shopping Plaza,Convenience Store,Café,Pizza Place,Construction & Landscaping


In [106]:
#Cluster 2

toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[0] + list(range(5, toronto_merged.shape[1]))]]

,PostalCode,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
32,M1J,2,Playground,Yoga Studio,Eastern European Restaurant,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Electronics Store
83,M4T,2,Trail,Playground,Yoga Studio,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant
85,M1V,2,Playground,Park,Coffee Shop,Yoga Studio,Drugstore,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop


In [107]:
#Cluster 3

toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[0] + list(range(5, toronto_merged.shape[1]))]]

,PostalCode,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
45,M2L,3,Cafeteria,Dumpling Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Eastern European Restaurant,College Stadium


In [108]:
#Cluster 4

toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[0] + list(range(5, toronto_merged.shape[1]))]]

,PostalCode,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
7,M3B,4,Café,Gym / Fitness Center,Japanese Restaurant,Caribbean Restaurant,Basketball Court,Dumpling Restaurant,Dog Run,Doner Restaurant,Donut Shop,Drugstore
10,M6B,4,Pub,Park,Sushi Restaurant,Asian Restaurant,Japanese Restaurant,Yoga Studio,Discount Store,Dog Run,Doner Restaurant,Donut Shop
12,M1C,4,Moving Target,Bar,Dumpling Restaurant,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Yoga Studio,Diner
14,M4C,4,Bus Stop,Curling Ice,Pharmacy,Park,Skating Rink,Video Store,Beer Store,Cosmetics Shop,Dance Studio,Asian Restaurant
16,M6C,4,Field,Trail,Dog Run,Hockey Arena,Discount Store,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant
18,M1E,4,Electronics Store,Pizza Place,Intersection,Breakfast Spot,Medical Center,Rental Car Location,Mexican Restaurant,Yoga Studio,Donut Shop,Dog Run
19,M4E,4,Trail,Health Food Store,Neighborhood,Pub,Yoga Studio,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop
25,M6G,4,Grocery Store,Café,Park,Nightclub,Diner,Italian Restaurant,Baby Store,Restaurant,Candy Store,Coffee Shop
26,M1H,4,Hakka Restaurant,Bakery,Bank,Athletics & Sports,Thai Restaurant,Caribbean Restaurant,Fried Chicken Joint,Discount Store,Doner Restaurant,Donut Shop
31,M6H,4,Pharmacy,Supermarket,Bakery,Athletics & Sports,Middle Eastern Restaurant,Music Venue,Park,Portuguese Restaurant,Café,Brewery
